In [28]:
#.env에서 API키 가져오기
import os
from dotenv import load_dotenv
load_dotenv('.env')
CLIENT_ID = os.getenv('X_NAVER_CLIENT_ID')
CLIENT_SECRET = os.getenv('X_NAVER_CLIENT_SECRET')


In [ ]:
## 지금 작업중 (2/26 기준)

In [41]:
import requests
import time
import json
import os

# 네이버 API 환경변수 가져오기
CLIENT_ID = os.getenv('X_NAVER_CLIENT_ID')
CLIENT_SECRET = os.getenv('X_NAVER_CLIENT_SECRET')

# ✅ 검색할 소설 키워드 목록
query_list = ["한국 소설", "일본 소설", "영미 소설"]

# 📌 API 요청 함수
def search_books(query, start=1, display=100):
    url = "https://openapi.naver.com/v1/search/book.json"
    headers = {
        "X-Naver-Client-Id": CLIENT_ID,
        "X-Naver-Client-Secret": CLIENT_SECRET
    }
    params = {
        "query": query,
        "display": display,  # 최대 100개 가져오기
        "start": start,  # 페이징 처리
        "sort": "sim"
    }

    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        return response.json()["items"]
    else:
        print(f"⚠️ API 요청 실패 ({response.status_code}): {query}, start={start}")
        return []

# 📌 전체 데이터 저장할 리스트
all_books = []
exclude_keywords = ["단편", "소설집", "단편선", "앤솔러지", "모음집", "옴니버스","자서전","시집","에세이","수필"]

# ✅ 여러 검색어로 데이터 수집
for query in query_list:
    for start in range(1, 1000, 100):  # 1000권 이상 확보 시도
        books = search_books(query, start)
        if not books:
            break  # 더 이상 결과 없음
        
        # 🔹 단편 소설 제외 (제목 또는 설명에 특정 키워드 포함 시 필터링)
        filtered_books = []
        for book in books:
            title = book["title"]
            description = book["description"]
            author = book.get("author", "알 수 없음")  # 🔹 작가 정보 가져오기
            
            # 📌 제외 키워드가 제목이나 설명에 포함되지 않은 경우만 저장
            if not any(keyword in title or keyword in description for keyword in exclude_keywords):
                filtered_books.append({
                    "title": title.split("(")[0].strip(),  # 🔹 제목에서 부가 정보 제거
                    "description": description.replace(".", ".\n"),  # 🔹 줄바꿈 추가
                    "author": author,  # 🔹 작가 정보 포함
                    "summary": ""  # 🔹 줄거리(요약)는 비워둠
                })

        all_books.extend(filtered_books)
        time.sleep(0.5)  # API 호출 제한 방지를 위해 대기

# ✅ 중복 제거 (책 제목 기준)
unique_books = {book["title"]: book for book in all_books}.values()

# 📌 JSON 파일로 저장
output_file = "filtered_novels.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(list(unique_books), f, ensure_ascii=False, indent=4)

print(f"✅ 총 {len(unique_books)}권의 장편 소설 데이터가 '{output_file}'에 저장되었습니다!")


✅ 총 1637권의 장편 소설 데이터가 'filtered_novels.json'에 저장되었습니다!


In [39]:
import json
import random

# 📌 1. 전체 1681권 데이터 로드
with open("filtered_novels.json", "r", encoding="utf-8") as f:
    all_books = json.load(f)

# ✅ 2. 랜덤으로 200권 선택 (라벨링용)
random.shuffle(all_books)  # 랜덤 셔플
labeled_books = all_books[:200]  # 앞에서 200권 선택

# ✅ 3. 나머지 1481권 (모델 자동 요약용)
unlabeled_books = all_books[200:]

# 📌 4. 라벨링 데이터 저장
with open("labeling_novels.json", "w", encoding="utf-8") as f:
    json.dump(labeled_books, f, ensure_ascii=False, indent=4)

# 📌 5. 모델 요약용 데이터 저장
with open("unlabeled_books.json", "w", encoding="utf-8") as f:
    json.dump(unlabeled_books, f, ensure_ascii=False, indent=4)

print(f"✅ 랜덤 200권 라벨링 완료! ('labeling_novels.json' 저장)")
print(f"✅ 나머지 1481권 모델 요약 준비 완료! ('unlabeled_books.json' 저장)")


✅ 랜덤 200권 라벨링 완료! ('labeling_novels.json' 저장)
✅ 나머지 1481권 모델 요약 준비 완료! ('unlabeled_books.json' 저장)


In [ ]:
###################################################################

In [27]:
import json
from rank_bm25 import BM25Okapi
import re

# ✅ JSON 파일 로드 (도서 데이터)
with open("novels.json", "r", encoding="utf-8") as f:
    books_data = json.load(f)

# ✅ 텍스트 전처리 (소문자 변환 + 불필요한 기호 제거)
def preprocess_text(text):
    text = text.lower()  # 소문자로 변환
    text = re.sub(r"[^\w\s]", "", text)  # 특수문자 제거
    return text

# ✅ BM25에 사용할 책 줄거리 데이터 준비
book_summaries = [preprocess_text(book.get("description", "")) for book in books_data]
tokenized_books = [summary.split() for summary in book_summaries]

# ✅ BM25 모델 생성
bm25 = BM25Okapi(tokenized_books)

# ✅ 사용자 입력 (검색할 줄거리)
query = "수학 교사가 전남편을 살해한 짝사랑하던 여자를 감싸기 위해 알리바이를 조작하는 소설"
tokenized_query = preprocess_text(query).split()

# ✅ BM25로 유사도 점수 계산
scores = bm25.get_scores(tokenized_query)

# ✅ 가장 유사한 상위 5개 책 찾기
top_n = 5
top_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:top_n]

# ✅ 검색 결과 출력
print("\n🔍 검색 결과 (Top 5):")
for idx in top_indices:
    book = books_data[idx]
    print(f"📖 제목: {book.get('title', '제목 없음')}")
    print(f"📜 줄거리: {book.get('description', '줄거리 없음')}")
    print(f"🔗 링크: {book.get('link', '링크 없음')}\n")



🔍 검색 결과 (Top 5):
📖 제목: 흔한 일들
📜 줄거리: 연쇄살인사건의 비밀을 파헤치는 한국적인 크라임 스릴러!

범죄 전문 기자 출신 작가 신재형이 쓴 한국형 크라임 스릴러 『흔한 일들』. 연쇄살인사건을 배경으로 최고의 프로파일러와 현장을 조작하는 범인의 치열한 두뇌 싸움을 그리고 있다. 서울 보광동에서 발생한 일가족 살인사건. 피해자는 여덟 살 딸과 그 어머니, 유력한 용의자는 현장을 배회하던 이웃집 여자 차아령. 6개월 전 연쇄살인범을 검거한 서울청 현장감식반 최재준 형사가 사건에 투입된다. 그는 차아령을 심문하던 중 피해자의 남편이 알리바이를 속인 사실을 알아내고 재빨리 탐문에 돌입하지만, 남편은 숨진 채 발견된다. 사체의 훼손상태를 살펴보던 최재준 형사는 이 사건이 자신을 겨낭하고 있다는 것을 깨닫게 되는데….
🔗 링크: https://search.shopping.naver.com/book/catalog/32506472270

📖 제목: 다감 선생님은 아이들이 싫다(리커버 개정판) (공민철 연작소설)
📜 줄거리: 타고난 명탐정 초등학교 교사가 아이들과 함께 겪는 성장통 이야기

좀처럼 아이들에게 마음을 열지 못하던 초등학교 교사 다감은 학교 내 자살 미수 사건을 해결하면서 6학년 1반의 담임이 된다. 이후로도 살인 사건, 방화 사건, 학교 폭력 등 아이들과 관련되어 예사롭지 않은 일들을 겪게 되는데, 진실을 알기 위해 사건을 하나둘 해결해 나가며 자신의 내면에서 아이들의 존재가 조금씩 커지는 것을 느끼는데…….
🔗 링크: https://search.shopping.naver.com/book/catalog/39849318619

📖 제목: 이방인 : 보성어부 살인 사건
📜 줄거리: 2007년! 녹차로 유명한 보성에서 충격적인 사건이 발생했다.
70대 어부가 성 욕구를 채우기 위해 자신의 배에 탄 젊은 남녀 대학생을 살해한 사건이었다.
노인은 2명의 젊은 여자를 같은 방법 같은 목적으로 추가 살해했다.
사람들은 노인의 엽기적 행각에 울분을 터뜨렸

In [34]:
from sentence_transformers import SentenceTransformer, util
import json

# ✅ 사전 훈련된 Sentence Embedding 모델 로드
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# ✅ JSON 파일 로드 (도서 데이터)
with open("novels.json", "r", encoding="utf-8") as f:
    books_data = json.load(f)

# ✅ 책 줄거리 리스트 생성
book_summaries = [book.get("description", "") for book in books_data]

# ✅ 책 줄거리들을 벡터로 변환
book_vectors = model.encode(book_summaries, convert_to_tensor=True)

# ✅ 사용자 입력
query = "어떤 수학 천재가 살인을 저지른 여자를 도와주려는 소설"
query_vector = model.encode(query, convert_to_tensor=True)

# ✅ 코사인 유사도 계산
similarities = util.pytorch_cos_sim(query_vector, book_vectors)[0]

# ✅ 가장 유사한 상위 5개 책 찾기
top_n = 5
top_indices = similarities.argsort(descending=True)[:top_n]

# ✅ 검색 결과 출력
print("\n🔍 검색 결과 (Top 5):")
for idx in top_indices:
    book = books_data[idx]
    print(f"📖 제목: {book.get('title', '제목 없음')}")
    print(f"📜 줄거리: {book.get('description', '줄거리 없음')}")
    print(f"🔗 링크: {book.get('link', '링크 없음')}\n")


🔍 검색 결과 (Top 5):
📖 제목: 경범죄 전문 흥신소
📜 줄거리: 대한민국의 경범죄 치한률 증가. 경찰이 해결 못해주는 사건을 맡아주는 경범죄 전문 흥신소 등장
🔗 링크: https://search.shopping.naver.com/book/catalog/44021367621

📖 제목: 사랑오운 아가미
📜 줄거리: 세상이 부정하더라도 모든 사랑은 빛나기에
빛나는 사랑을 하고 계신 분들께 이 책을 바칩니다
🔗 링크: https://search.shopping.naver.com/book/catalog/52782956485

📖 제목: 카페 홈즈의 마지막 사랑(큰글씨책)
📜 줄거리: 추리소설 작가들의 아지트인 ‘카페 홈즈’
그곳을 무대로 펼쳐지는 여섯 작가의 독특한 여섯 가지 이야기
🔗 링크: https://search.shopping.naver.com/book/catalog/32441033830

📖 제목: 사라진 피리 (고대 도시에서의 추적)
📜 줄거리: 고대 도시 경주에서 펼쳐지는 5일간의 모험
🔗 링크: https://search.shopping.naver.com/book/catalog/51896716626

📖 제목: 트윈 풀 호텔의 살인 사건
📜 줄거리: 포문 출판 노원 작가의 대표작 ‘어릿광대 저택의 살인 사건’에서 활약한 아일랜과 뉴윈의 두 번째 사건 해결서
🔗 링크: https://search.shopping.naver.com/book/catalog/45813431623



In [39]:
from sentence_transformers import SentenceTransformer, util
import json
import numpy as np
from rank_bm25 import BM25Okapi
import re

# ✅ 더 강력한 모델 사용 ('multi-qa-mpnet-base-dot-v1')
model = SentenceTransformer("sentence-transformers/multi-qa-mpnet-base-dot-v1")

# ✅ JSON 파일 로드 (도서 데이터)
with open("novels.json", "r", encoding="utf-8") as f:
    books_data = json.load(f)

# ✅ 데이터 정제 및 줄거리 리스트 생성 (너무 짧은 줄거리는 제외)
filtered_books = [book for book in books_data if len(book.get("description", "")) > 20]

# ✅ 책 줄거리 리스트 생성
book_summaries = [book.get("description", "") for book in filtered_books]

# ✅ 1️⃣ BM25 기반 검색 수행 (불필요한 특수문자 제거)
tokenized_summaries = [re.sub(r"\W+", " ", summary.lower()).split() for summary in book_summaries]
bm25 = BM25Okapi(tokenized_summaries)

# ✅ 사용자 입력 (검색어)
query = "여자주인공이 도시락집 직원인데 전남편을 살해했어. 이웃집 수학 천재가 그 여자를 짝사랑해서 그 여자를 위해 알리바이를 만들어주고 자기자신은 죄를 뒤집어쓰려고 해. 그 소설이 뭐지?"

# ✅ BM25 검색 적용 (Top 50개 선택)
query_tokens = re.sub(r"\W+", " ", query.lower()).split()
bm25_scores = bm25.get_scores(query_tokens)
top_n_bm25 = np.argsort(bm25_scores)[::-1][:50]  # ✅ BM25 기반 Top 50개 선택

# ✅ 2️⃣ 선택된 Top 50개에 대해 SentenceTransformer 유사도 비교
selected_books = [book_summaries[i] for i in top_n_bm25]
selected_vectors = model.encode(selected_books, convert_to_tensor=True)

query_vector = model.encode(query, convert_to_tensor=True)
similarities = util.pytorch_cos_sim(query_vector, selected_vectors)[0].cpu().numpy()

# ✅ SentenceTransformer 유사도 점수를 활용하여 최종 Top 5 정렬
top_n = 5
sorted_indices = np.argsort(similarities)[::-1][:top_n]

# ✅ 검색 결과 출력
print("\n🔍 검색 결과 (Top 5):")
for idx in sorted_indices:
    book = filtered_books[top_n_bm25[idx]]
    print(f"📖 제목: {book.get('title', '제목 없음')}")
    print(f"📜 줄거리: {book.get('description', '줄거리 없음')}")
    print(f"🔗 링크: {book.get('link', '링크 없음')}\n")




🔍 검색 결과 (Top 5):
📖 제목: 옥중금낭, 첫날밤의 살인사건
📜 줄거리: 과거를 보러 서울에 올라온 장한응은 아이들의 장난으로 곤경에 처한 맹인 점쟁이를 구해 준다. 점쟁이는 그 사례로 점을 쳐 주는데, 세 가지 액운을 예언한다. 불에 타 죽을 운수와 물에 빠져 죽을 운수, 옥에 갇혀 죽을 운수가 그것이다. 《옥중금낭》은 장한응에게 닥쳐오는 세 가지 액운을 어떻게 극복할 것인가에 대한 긴장을 담고 있다. 중심은 신혼 첫날밤 벌어진 살인사건이다. 알리바이를 입증 못해 옥에 갇힌 장한응. 진짜 범인은 누구인가?
🔗 링크: https://search.shopping.naver.com/book/catalog/51398618626

📖 제목: 옥중금낭, 첫날밤의 살인사건 (큰글자책)
📜 줄거리: 과거를 보러 서울에 올라온 장한응은 아이들의 장난으로 곤경에 처한 맹인 점쟁이를 구해 준다. 점쟁이는 그 사례로 점을 쳐 주는데, 세 가지 액운을 예언한다. 불에 타 죽을 운수와 물에 빠져 죽을 운수, 옥에 갇혀 죽을 운수가 그것이다. 《옥중금낭》은 장한응에게 닥쳐오는 세 가지 액운을 어떻게 극복할 것인가에 대한 긴장을 담고 있다. 중심은 신혼 첫날밤 벌어진 살인사건이다. 알리바이를 입증 못해 옥에 갇힌 장한응. 진짜 범인은 누구인가?
🔗 링크: https://search.shopping.naver.com/book/catalog/51408008630

📖 제목: 꿈에서 온 그녀
📜 줄거리: “꿈에서 본 여자를 현실에서 만났다면, 꿈에서 일어난 사건도 현실에서 일어날 수 있는 게 아닌가?”꿈과 현실의 경계를 넘나드는 섬세한 서사와 심리적 긴장감이 돋보이는 문학적 서스펜스.
주인공 함지훈은 태어나는 순간 어머니를 잃고, 아버지의 냉대 속에서 상처와 죄책감을 안고 성장했다. 어느 날, 꿈속에서 지훈은 광기 어린 웃음을 짓는 여인을 만나고, 현실에서 그녀와 마주한 뒤 삶이 예측할 수 없는 방향으로 흔들리기 시작한다. 연이어 벌어지는 비극 속에서 지훈은 믿음과 의심, 사랑과 

In [45]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
import json

# 1️⃣ BAAI/bge-m3 모델 로드 (정규화 필수)
model = SentenceTransformer("BAAI/bge-m3")
model.max_seq_length = 512

# 2️⃣ novels.json 파일 로드
with open("novels.json", "r", encoding="utf-8") as f:
    novels = json.load(f)

# 3️⃣ 소설 줄거리를 벡터로 변환 (✅ 정규화 추가)
descriptions = [novel["description"] for novel in novels]
embeddings = model.encode(descriptions, normalize_embeddings=True, convert_to_numpy=True)

# 4️⃣ FAISS 코사인 유사도 검색을 위한 Index 생성
d = embeddings.shape[1]  # 1024차원
index = faiss.IndexFlatIP(d)  # ✅ Inner Product (IP) 기반 인덱스
faiss.normalize_L2(embeddings)  # ✅ 벡터 정규화 필수
index.add(embeddings)

# 5️⃣ 인덱스 저장 (재사용 가능)
faiss.write_index(index, "novel_index_cosine.faiss")

# 6️⃣ JSON 파일로 소설 데이터 저장 (검색 시 사용)
with open("novel_data.json", "w", encoding="utf-8") as f:
    json.dump(novels, f, ensure_ascii=False, indent=4)

print("✅ FAISS 코사인 유사도 인덱스 저장 완료!")



✅ FAISS 코사인 유사도 인덱스 저장 완료!


In [50]:
import faiss
import numpy as np
import json
from sentence_transformers import SentenceTransformer

# 1️⃣ BAAI/bge-m3 모델 로드
model = SentenceTransformer("BAAI/bge-m3")
model.max_seq_length = 512

# 2️⃣ FAISS 인덱스 로드
index = faiss.read_index("novel_index_cosine.faiss")

# 3️⃣ 소설 데이터 로드
with open("novel_data.json", "r", encoding="utf-8") as f:
    novels = json.load(f)

# 4️⃣ 검색 함수 정의
def search_similar_novels(query, top_k=20):  # ✅ top_k=20 으로 확장
    query_embedding = model.encode([query], normalize_embeddings=True, convert_to_numpy=True)
    faiss.normalize_L2(query_embedding)  # ✅ 검색 전 벡터 정규화 필수
    
    distances, indices = index.search(query_embedding, top_k)
    
    results = []
    for i, idx in enumerate(indices[0]):
        if idx < len(novels):
            results.append({
                "rank": i+1,
                "title": novels[idx]["title"],
                "link": novels[idx]["link"],
                "description": novels[idx]["description"],
                "similarity": float(distances[0][i])
            })
    
    return results

# 5️⃣ 테스트 검색 (예제)
query = "어떤 여자가 전남편을 살해했어. 그 여자를 짝사랑해왔던 이웃집 수학교사가 그 여자를 위해 알리바이를 만들어주고 자기자신이 죄를 뒤집어쓰려고 해"
results = search_similar_novels(query)

# 6️⃣ 검색 결과 출력
for res in results[:5]:  # ✅ 상위 5개 결과만 출력
    print(f"{res['rank']}. {res['title']} ({res['similarity']:.4f})")
    print(f"🔗 {res['link']}")
    print(f"📖 {res['description'][:100]}...\n")


1. 누명 (0.5345)
🔗 https://search.shopping.naver.com/book/catalog/33434700728
📖 막대한 자산을 가진 의붓어머니를 살해했다는 죄로 복역중인 쟈코는 강력히 무죄를 주장하나 결국 옥사한다. 그러나 그에게는 철벽 알리바이가 있었다! 그렇다면 진짜 범인은 누구인가? 인...

2. 그날로 다시 돌아가 널 살리고 싶어 (우대경 장편소설) (0.5236)
🔗 https://search.shopping.naver.com/book/catalog/39072822623
📖 촉법소년에게 아들을 잃은 은서. 그녀에게 건네진 낡은 일기장.
그 일기를 통해 시작된 타임슬립, 그리고 복수.

“내가 과거로 가면 미래가 달라지기라도 하니? 우리 아들이 살아 돌...

3. 중간지점의 집 (0.5209)
🔗 https://search.shopping.naver.com/book/catalog/32436143223
📖 오두막에서 한 남자가 살해된다. 이 남자는 누구이며 왜 살해되었는가? 아름다움과 추함, 빈곤과 부유, 혼자이면서 두 사람의 피해자이기도 한 이중성 알리바이 상쾌한 추리가 질주, 질...

4. 킬에이저 (0.5202)
🔗 https://search.shopping.naver.com/book/catalog/49644211622
📖 ‘올해 가장 주목받는 한국의 여성 리더 10인’에 선정될 정도로 대한민국을 대표하는 1세대 프로파일러 강해수. 야망 때문에 남편과 이혼한 후, 아들을 명문 고등학교에 보낸다. 그러...

5. 종이학 살인사건 (0.5186)
🔗 https://search.shopping.naver.com/book/catalog/35326331623
📖 아버지의 시신에서 암호를 발견한 그날,
멈췄던 연쇄살인이 다시 시작됐다.

의사인 치하야는 어머니의 죽음 후 멀어진 아버지와의 관계를 회복하지 못한 채, 아버지마저 암으로 떠나보내...



In [55]:
import faiss
import numpy as np
import json
from sentence_transformers import SentenceTransformer

# 1️⃣ BAAI/bge-m3 모델 로드
model = SentenceTransformer("BAAI/bge-m3")
model.max_seq_length = 512

# 2️⃣ FAISS 인덱스 로드
index = faiss.read_index("novel_index_cosine.faiss")

# 3️⃣ 소설 데이터 로드
with open("novel_data.json", "r", encoding="utf-8") as f:
    novels = json.load(f)

# 4️⃣ 검색 함수 정의
def search_similar_novels(query, top_k=20):  # ✅ 검색 개수 증가
    # ✅ 쿼리에 "질문:" 추가
    query_embedding = model.encode(["질문: " + query], normalize_embeddings=True, convert_to_numpy=True)
    faiss.normalize_L2(query_embedding)  # ✅ 벡터 정규화 추가

    distances, indices = index.search(query_embedding, top_k)

    results = []
    for i, idx in enumerate(indices[0]):
        if idx < len(novels):
            results.append({
                "rank": i+1,
                "title": novels[idx]["title"],
                "link": novels[idx]["link"],
                "description": novels[idx]["description"],
                "similarity": float(distances[0][i])
            })

    return results

# 5️⃣ 테스트 검색
query = """
도시락 가게에서 일하는 여성이 있습니다. 그녀는 전남편을 우발적으로 살해했으며, 
그녀의 이웃인 수학 천재 남성은 그녀를 오랫동안 짝사랑해 왔습니다. 
이 남성은 그녀를 돕기 위해 완벽한 알리바이를 만들어주고, 
자신이 대신 죄를 뒤집어쓰려고 합니다. 이 이야기가 담긴 소설의 제목은 무엇인가요?
"""

results = search_similar_novels(query)

# 6️⃣ 검색 결과 출력
for res in results[:5]:  # ✅ 상위 5개 결과만 출력
    print(f"{res['rank']}. {res['title']} ({res['similarity']:.4f})")
    print(f"🔗 {res['link']}")
    print(f"📖 {res['description'][:100]}...\n")


1. 시간의 조각 (0.5620)
🔗 https://search.shopping.naver.com/book/catalog/52522355605
📖 김동원│로버트
조금 특별한 친구 ‘로버트’. 어눌한 말투와 무표정한 얼굴, 특정 분야에 몰두해 있고 말의 맥락을 파악할 줄 모르며 상대방의 기분이나 상황을 고려하지 않고 자신의 관...

2. 용의자 X의 헌신 (갈릴레오 시리즈 3) (0.5569)
🔗 https://search.shopping.naver.com/book/catalog/32485526273
📖 정교한 살인수식에 도전하는 천재 물리학자의 집요한 추적이 시작된다!

《동급생》, 《백야행》의 작가 히가시노 게이고의 장편소설. 2006년 제134회 나오키 상 수상작이다. 일본 ...

3. 미스터리 살인 사건 7+1 (0.5567)
🔗 https://search.shopping.naver.com/book/catalog/42654561620
📖 미스터리 살인 사건에는 규칙이 있다.
반드시 피해자, 용의자, 수사관이 있을 것.

수학 교수인 그랜트 맥알리스터가 그 모든 규칙을 세우고 연구한 다음 완벽한 일곱 편의 이야기를 ...

4. 한밤중의 마리오네트 (0.5540)
🔗 https://search.shopping.naver.com/book/catalog/51896750620
📖 내가 살려낸 환자가 약혼자를 죽인 연쇄살인마라면?
토막 난 시신 옆에서 발견된 그는…
무고한 피해자인가, 아니면 끔찍한 살인마인가?!

하룻밤 사이에 사람을 토막 내는 엽기살인마,...

5. 용의자 X의 헌신 (제134회 나오키상 수상작, 갈릴레오 시리즈 3) (0.5519)
🔗 https://search.shopping.naver.com/book/catalog/32463541483
📖 정교한 살인수식에 도전하는 천재 물리학자의 집요한 추적이 시작된다!

히가시노 게이고 문학의 정수로 일컬어지는 추리 소설 『용의자 X의 헌신』. 일본 문학 전문 번역가 양억관이 자...



In [56]:
import faiss
import numpy as np
import json
from sentence_transformers import SentenceTransformer

# ✅ 모델 로드
model = SentenceTransformer("BAAI/bge-m3")
model.max_seq_length = 512

# ✅ 소설 데이터 로드
with open("novels.json", "r", encoding="utf-8") as f:
    novels = json.load(f)

# ✅ 소설 줄거리 벡터 변환
descriptions = [novel["description"] for novel in novels]
embeddings = model.encode(descriptions, normalize_embeddings=True, convert_to_numpy=True)

# ✅ 벡터 정규화 (코사인 유사도 기반 검색용)
faiss.normalize_L2(embeddings)

# ✅ FAISS 인덱스 생성 (Inner Product 기반)
d = embeddings.shape[1]  # 벡터 차원
index = faiss.IndexFlatIP(d)  # IP (Inner Product) 방식 사용
index.add(embeddings)  # 벡터 추가

# ✅ FAISS 인덱스 저장
faiss.write_index(index, "novel_index_cosine.faiss")

print("✅ FAISS 인덱스 생성 완료! (코사인 유사도 기반)")


✅ FAISS 인덱스 생성 완료! (코사인 유사도 기반)


In [60]:
import faiss
import numpy as np
import json
from sentence_transformers import SentenceTransformer

# 1️⃣ BAAI/bge-m3 모델 로드
model = SentenceTransformer("BAAI/bge-m3")
model.max_seq_length = 512  # 입력 문장 길이 제한

# 2️⃣ FAISS 인덱스 로드
index = faiss.read_index("novel_index.faiss")

# 3️⃣ 소설 데이터 로드
with open("novel_data.json", "r", encoding="utf-8") as f:
    novels = json.load(f)

# ✅ 🔥 다양한 쿼리 변형 함수 추가
def generate_query_variants(query):
    """여러 가지 방식으로 쿼리를 변형하여 검색 정확도를 높임"""
    return [
        f"이 소설은 {query} 라는 이야기를 다룬다.",
        f"이 이야기의 핵심 키워드는 {query} 이다.",
        f"이 책은 {query} 를 중심으로 전개된다.",
        f"이 소설에서 주요 인물들은 {query} 속에서 관계를 맺는다.",
        f"줄거리는 {query} 를 기반으로 진행된다."
    ]

# ✅ 🔥 다중 쿼리 검색 수행 함수
def search_similar_novels_multi(query, top_k=10):
    """다양한 쿼리를 생성하고, 다중 검색 후 최적의 결과를 반환"""
    query_variants = generate_query_variants(query)  # 여러 개의 변형된 쿼리 생성
    all_results = {}

    for q in query_variants:
        query_embedding = model.encode([q], normalize_embeddings=True, convert_to_numpy=True)
        faiss.normalize_L2(query_embedding)

        distances, indices = index.search(query_embedding, top_k)

        for i, idx in enumerate(indices[0]):
            if idx < len(novels):
                title = novels[idx]["title"]
                similarity = float(distances[0][i])

                # 동일한 책이 여러 번 등장하면 가장 높은 점수 유지
                if title in all_results:
                    all_results[title]["similarity"] = max(all_results[title]["similarity"], similarity)
                else:
                    all_results[title] = {
                        "rank": len(all_results) + 1,
                        "title": title,
                        "link": novels[idx]["link"],
                        "description": novels[idx]["description"],
                        "similarity": similarity
                    }

    # ✅ 점수 순으로 정렬
    sorted_results = sorted(all_results.values(), key=lambda x: x["similarity"], reverse=True)
    return sorted_results[:5]

# ✅ 테스트 실행 (쿼리 입력)
query = """
일본소설이야. 도시락가게에서 일하는 여자가 있는데, 전남편을 우발적으로 살해했다.
그 여자를 짝사랑하던 이웃의 수학교사가 그 여자를 돕기 위해 완벽한 알리바이를 만들어주고,
자신이 대신 죄를 뒤집어쓰려고 한다. 이 소설의 제목은 무엇인가요?
"""

# 🔍 최종 검색 실행
results = search_similar_novels_multi(query)

# ✅ 검색 결과 출력
for res in results:
    print(f"{res['rank']}. {res['title']} ({res['similarity']:.4f})")
    print(f"🔗 {res['link']}")
    print(f"📖 {res['description'][:100]}...\n")



15. 도련님 (미니북) (0.8778)
🔗 https://search.shopping.naver.com/book/catalog/32436072624
📖 일본의 셰익스피어 나쓰메 소세키를 인기 작가 반열에 올린 작품이다. 사회 부조리와 기회주의적 인간에 대한 비판을 담고 있다. 비판 의식이 가득한 소설이지만 무겁게 이야기를 끌어가는...

14. 수인번호 1004 (죽은 자는 말이 없지만, 완전범죄란 있을 수 없다!) (0.8767)
🔗 https://search.shopping.naver.com/book/catalog/41203603620
📖 죽은 자는 말이 없지만 완전범죄란 있을 수 없다. 이 말이 가능한 것은 범인과 진실을 쫓는 이들의 부단한 노력과 집념이 있기 때문일 것이다. 실화를 바탕으로 쓰인 이 미스터리 소설...

13. 나오미와 가나코 (오쿠다 히데오 장편소설) (0.8760)
🔗 https://search.shopping.naver.com/book/catalog/32436121677
📖 우리는 오늘 남편을 죽였다!

오쿠다 히데오의 장편소설 『나오미와 가나코』. 그동안의 스타일에서 벗어나 고도의 서스펜스 스타일로 새롭게 변신을 시도한 오쿠다 히데오. 저자 자신도 ...

7. 테사를 찾아서(큰글자도서) (0.8738)
🔗 https://search.shopping.naver.com/book/catalog/50402736623
📖 불행한 어린 시절을 보내며 성장했으면서도 행복한 삶에 대한 희망과 기대를 놓지 않는 테사. 그녀는 자신을 구원해 줄 새 인생을 꿈꾸며 지옥 같은 이전 삶에서 탈출을 시도한다. 끊임...

8. 테사를 찾아서 (0.8738)
🔗 https://search.shopping.naver.com/book/catalog/49893111618
📖 불행한 어린 시절을 보내며 성장했으면서도 행복한 삶에 대한 희망과 기대를 놓지 않는 테사. 그녀는 자신을 구원해 줄 새 인생을 꿈꾸며 지옥 같은 이전 삶에서 탈출을 시도한다. 끊임...


In [61]:

import faiss
import numpy as np
import json
from sentence_transformers import SentenceTransformer

# ✅ 모델 로드 (BAAI/bge-m3)
model = SentenceTransformer("BAAI/bge-m3")
model.max_seq_length = 512

# ✅ 소설 데이터 로드
with open("novels.json", "r", encoding="utf-8") as f:
    novels = json.load(f)

# ✅ 🔥 새로운 임베딩 전략: 제목 + 작가 + 줄거리 포함
def create_embedding_text(novel):
    return f"제목: {novel['title']}. 저자: {novel['author']}. 내용: {novel['description']}"

# ✅ 🔥 벡터 변환 (제목 + 저자 + 설명 포함)
descriptions = [create_embedding_text(novel) for novel in novels]
embeddings = model.encode(descriptions, normalize_embeddings=True, convert_to_numpy=True)

# ✅ 벡터 정규화 (코사인 유사도 기반 검색용)
faiss.normalize_L2(embeddings)

# ✅ FAISS 인덱스 생성 (Inner Product 기반)
d = embeddings.shape[1]  # 벡터 차원
index = faiss.IndexFlatIP(d)  # IP (Inner Product) 방식 사용
index.add(embeddings)  # 벡터 추가

# ✅ FAISS 인덱스 저장
faiss.write_index(index, "novel_index_cosine_v2.faiss")

print("✅ 새로운 FAISS 인덱스 생성 완료! (제목+저자+설명 포함)")


✅ 새로운 FAISS 인덱스 생성 완료! (제목+저자+설명 포함)


In [62]:
import faiss
import numpy as np
import json
from sentence_transformers import SentenceTransformer

# ✅ 모델 로드 (BAAI/bge-m3)
model = SentenceTransformer("BAAI/bge-m3")
model.max_seq_length = 512

# ✅ FAISS 인덱스 로드 (업데이트된 버전 사용)
index = faiss.read_index("novel_index_cosine_v2.faiss")

# ✅ 소설 데이터 로드
with open("novel_data.json", "r", encoding="utf-8") as f:
    novels = json.load(f)

# ✅ 🔥 사용자 질문을 소설 설명처럼 변환
def transform_query(query):
    return f"이 소설의 제목과 주요 내용은 다음과 같습니다: {query}"

# ✅ 🔥 검색 함수 (쿼리 변형 + 벡터 변환)
def search_similar_novels(query, top_k=10):
    query_text = transform_query(query)  # ✅ 쿼리 변형 적용
    query_embedding = model.encode([query_text], normalize_embeddings=True, convert_to_numpy=True)
    faiss.normalize_L2(query_embedding)  # ✅ 벡터 정규화 적용

    distances, indices = index.search(query_embedding, top_k)

    results = []
    for i, idx in enumerate(indices[0]):
        if idx < len(novels):
            results.append({
                "rank": i+1,
                "title": novels[idx]["title"],
                "author": novels[idx]["author"],
                "link": novels[idx]["link"],
                "description": novels[idx]["description"],
                "similarity": float(distances[0][i])
            })

    return results

# ✅ 테스트 실행 (쿼리 입력)
query = """
도시락가게에서 일하는 여자가 있는데, 전남편을 우발적으로 살해했다.
이웃의 수학교사가 그 여자를 짝사랑하며, 그 여자를 돕기 위해 완벽한 알리바이를 만들어주고,
자신이 대신 죄를 뒤집어쓰려고 한다. 이 소설의 제목은 무엇인가요?
"""

# 🔍 최종 검색 실행
results = search_similar_novels(query)

# ✅ 검색 결과 출력
for res in results:
    print(f"{res['rank']}. {res['title']} ({res['similarity']:.4f})")
    print(f"👨‍💻 {res['author']}")
    print(f"🔗 {res['link']}")
    print(f"📖 {res['description'][:100]}...\n")


1. 달콤한 살인 계획 (김서진 장편소설) (0.5658)
👨‍💻 김서진
🔗 https://search.shopping.naver.com/book/catalog/48242544621
📖 모든 것이 결핍된 여자가 세상을 향해 겨누는 뜨거운 칼끝
세계문학상 우수상 수상 작가 김서진, 10년 만의 신작

나 자신으로부터 도망치고 싶은 적이 있는가? 벗어날 수 없는 과거...

2. 도시락 (제30회 소설미학 신인소설상 장편소설부문 당선작) (0.5616)
👨‍💻 김선덕
🔗 https://search.shopping.naver.com/book/catalog/51342226802
📖 시대에 필요한 정신을 담아내고자 노력한 소설이다. 주인공이 재건축 중에 땅속에서 도시락을 발견했는데 부친 소유의 부동산 등기필증이 담겨있었다. 처음에는 이기적인 욕망으로 가득했지만...

3. 그녀가 마지막에 본 것은 (0.5607)
👨‍💻 마사키 도시카
🔗 https://search.shopping.naver.com/book/catalog/40827394639
📖 누구나 행복해야 할 크리스마스이브, 노숙인으로 추정되는 신원 미상의 중년 여성이 시신으로 발견된다. 딱딱한 콘크리트 바닥에 반듯하게 누워 있는 여성의 옷은 흐트러졌고 머리에는 둔기...

4. 한밤중의 마리오네트 (0.5558)
👨‍💻 치넨 미키토
🔗 https://search.shopping.naver.com/book/catalog/51896750620
📖 내가 살려낸 환자가 약혼자를 죽인 연쇄살인마라면?
토막 난 시신 옆에서 발견된 그는…
무고한 피해자인가, 아니면 끔찍한 살인마인가?!

하룻밤 사이에 사람을 토막 내는 엽기살인마,...

5. 치유를 파는 찻집 (0.5519)
👨‍💻 모리사와 아키오
🔗 https://search.shopping.naver.com/book/catalog/42074839626
📖 가슴이 뭉클해지는 감동 미스터리
기상천외한 방법으로 ‘치유를 파는’ 그녀에게
어느 날, 살인 예고장이 도착했다!



In [1]:
import faiss
import numpy as np
import json
from sentence_transformers import SentenceTransformer
from transformers import RagTokenizer, RagRetriever, RagSequenceForGeneration

# 1️⃣ RAG 모델 로드
rag_model_name = "facebook/rag-token-nq"

# 토크나이저 로드
tokenizer = RagTokenizer.from_pretrained(rag_model_name)

# FAISS 기반 RAG Retriever 설정 (RAG 모델 내에서 검색 기능 사용)
retriever = RagRetriever.from_pretrained(rag_model_name, index_name="exact", use_dummy_dataset=True)

# RAG 모델 로드
rag_model = RagSequenceForGeneration.from_pretrained(rag_model_name, retriever=retriever)

# 2️⃣ BAAI/bge-m3 모델 로드 (벡터 검색용)
embedding_model = SentenceTransformer("BAAI/bge-m3")
embedding_model.max_seq_length = 512

# 3️⃣ FAISS 인덱스 로드
index = faiss.read_index("novel_index_cosine.faiss")

# 4️⃣ 소설 데이터 로드
with open("novel_data.json", "r", encoding="utf-8") as f:
    novels = json.load(f)

# 5️⃣ 검색 함수 (FAISS + RAG)
def search_with_rag(query, top_k=5):
    # 1. FAISS로 관련 소설 검색
    query_embedding = embedding_model.encode([query], normalize_embeddings=True, convert_to_numpy=True)
    faiss.normalize_L2(query_embedding)

    # FAISS 검색 수행
    distances, indices = index.search(query_embedding, top_k)

    # 검색된 소설 정보 수집
    retrieved_novels = []
    for i, idx in enumerate(indices[0]):
        if idx < len(novels):
            retrieved_novels.append({
                "rank": i+1,
                "title": novels[idx]["title"],
                "description": novels[idx]["description"]
            })

    # 2. RAG 모델을 사용해 자연어 응답 생성
    novel_context = "\n".join([f"{novel['title']}: {novel['description']}" for novel in retrieved_novels])
    prompt = f"사용자가 입력한 질문: {query}\n\n아래는 검색된 소설 정보입니다:\n{novel_context}\n\n이 정보 기반으로 사용자의 질문에 가장 적절한 소설을 추천해주세요."

    # RAG 모델을 사용하여 답변 생성
    inputs = tokenizer(prompt, return_tensors="pt")
    generated_output = rag_model.generate(**inputs)
    response = tokenizer.decode(generated_output[0], skip_special_tokens=True)

    return response, retrieved_novels

# 6️⃣ 테스트 검색
query = """
도시락 가게에서 일하는 여성이 있습니다. 그녀는 전남편을 우발적으로 살해했으며, 
그녀의 이웃인 수학 천재 남성은 그녀를 오랫동안 짝사랑해 왔습니다. 
이 남성은 그녀를 돕기 위해 완벽한 알리바이를 만들어주고, 
자신이 대신 죄를 뒤집어쓰려고 합니다. 이 이야기가 담긴 소설의 제목은 무엇인가요?
"""

response, results = search_with_rag(query)

# 7️⃣ 결과 출력
print("📌 AI 추천 응답 (RAG 기반):")
print(response)

print("\n🔍 FAISS 검색된 소설 목록:")
for res in results:
    print(f"{res['rank']}. {res['title']}")


/opt/anaconda3/envs/hi/lib/python3.10/site-packages/transformers/models/bart/configuration_bart.py:176: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It m

: 

In [17]:
import shutil
import os

cache_dir = os.path.expanduser('~/.cache/huggingface/transformers/')
if os.path.exists(cache_dir):
    shutil.rmtree(cache_dir)
    print("Hugging Face 모델 캐시가 삭제되었습니다.")


In [ ]:
import requests
import json
import time
import os
from urllib.parse import quote

# 네이버 API 환경변수 가져오기
CLIENT_ID = os.getenv('X_NAVER_CLIENT_ID')
CLIENT_SECRET = os.getenv('X_NAVER_CLIENT_SECRET')

# 검색할 미스터리 소설 관련 키워드 (소설 제거)
query_list = [
    "미스터리", "추리", "서스펜스", "공포", 
    "스릴러", "심리 스릴러", "법정 스릴러", "탐정", "고전 추리",
    "해외 미스터리", "한국 미스터리"
]

# 전체 데이터 저장할 딕셔너리 (ISBN 기준 중복 방지)
book_dict = {}

# API 요청 함수 (부적절한 sort 값 수정)
def search_books(query, start=1, display=100, sort="sim"):
    url = "https://openapi.naver.com/v1/search/book.json"
    headers = {
        "X-Naver-Client-Id": CLIENT_ID,
        "X-Naver-Client-Secret": CLIENT_SECRET
    }
    params = {
        "query": query,  # ✅ URL 인코딩 적용
        "display": display,
        "start": start,
        "sort": sort  # ✅ "sim" 또는 "date"만 사용 (count 제거)
    }

    response = requests.get(url, headers=headers, params=params)
    response.encoding = 'utf-8'

    if response.status_code == 400:
        print(f"❌ API 요청 실패 (400): {query}, start={start}")
        print("📌 응답 메시지:", response.text)
        return []

    if response.status_code == 200:
        data = response.json()
        if data.get("total", 0) == 0:  # ✅ 검색 결과 없음 처리
            print(f"❌ '{query}' 검색 결과 없음. 건너뜀.")
            return []
        return data["items"]
    
    print(f"❌ API 요청 실패 ({response.status_code}): {query}, start={start}")
    return []

# 여러 검색어와 정렬 방식으로 데이터 수집 (sort 옵션 수정)
for query in query_list:
    for sort_option in ["sim", "date"]:  # ✅ "count" 제거
        for start in range(1, 1000, 100):  # 최대 1000권까지 확보
            books = search_books(query, start, sort=sort_option)
            if not books:
                break

            for book in books:
                isbn = book.get("isbn", "")
                if isbn and isbn not in book_dict:  # ISBN 기준 중복 제거
                    book_dict[isbn] = book
            
            time.sleep(1)  # ✅ 1초 대기 (API 제한 방지)

# JSON 파일로 저장
with open("mystery_books_10000.json", "w", encoding="utf-8") as f:
    json.dump(list(book_dict.values()), f, ensure_ascii=False, indent=4)  # ✅ UTF-8 저장 유지

print(f"✅ 총 {len(book_dict)}권의 미스터리 소설 데이터 저장 완료!")


✅ 총 5683권의 미스터리 소설 데이터 저장 완료!


In [21]:
import requests
import os
from urllib.parse import quote

CLIENT_ID = os.getenv('X_NAVER_CLIENT_ID')
CLIENT_SECRET = os.getenv('X_NAVER_CLIENT_SECRET')

query = "미스터리"  # 테스트용 키워드

url = "https://openapi.naver.com/v1/search/book.json"
headers = {
    "X-Naver-Client-Id": CLIENT_ID,
    "X-Naver-Client-Secret": CLIENT_SECRET
}
params = {
    "query": query,
    "sort": "sim"
}

# ✅ 실제 API 요청 URL 확인
request_url = f"{url}?query={params['query']}&display=10&sort=sim"
print(f"📌 실제 요청 URL: {request_url}")

response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()
    print("✅ API 응답 데이터:", data)
else:
    print(f"❌ API 요청 실패: {response.status_code}")
    print("응답 메시지:", response.text)


📌 실제 요청 URL: https://openapi.naver.com/v1/search/book.json?query=미스터리&display=10&sort=sim
✅ API 응답 데이터: {'lastBuildDate': 'Thu, 20 Feb 2025 17:23:35 +0900', 'total': 2776, 'start': 1, 'display': 10, 'items': [{'title': '당신이 누군가를 죽였다', 'link': 'https://search.shopping.naver.com/book/catalog/49073690634', 'image': 'https://shopping-phinf.pstatic.net/main_4907369/49073690634.20240713091011.jpg', 'author': '히가시노 게이고', 'discount': '17820', 'publisher': '북다', 'pubdate': '20240723', 'isbn': '9791170611561', 'description': '히가시노 게이고가 재현한\n황금시대 본격 미스터리\n\n히가시노 게이고 101번째 작품에서 미스터리의 원점으로!\n\n“미스터리란 어떤 소설인가? 라는 질문을 들었을 때 \n이런 소설이다, 라고 대답할 수 있는 작품입니다.”\n _히가시노 게이고\n\n일본 최고의 베스트셀러 작가 히가시노 게이고의 최신 장편소설『당신이 누군가를 죽였다』가 북다에서 출간되었다. 작품은 장르문학계의 거장인 작가가 101번째 작품을 맞아 추리소설의 원점으로 돌아가 ‘황금시대 미스터리’의 매력을 유감없이 발휘한 걸작으로 평단과 독자의 호평을 받고 있다. 1986년 발표된 『졸업』을 시작으로 장장 38년째 이어진 히가시노 게이고 미스터리의 정수인 〈가가 형사 시리즈〉 열두 번째 작품이기도 한 신작은, 2023년 출간 즉시 일본 서점 미스터리 판매 전체 1위를 석권하며 세월이 지나도 변치 않는 시리즈의 인기를 증명했다.\n『당신이 누군가를 죽였다』는 호화 별장지에 여름 휴

In [23]:
import json
import pandas as pd
import re

# 📌 저장된 JSON 파일 불러오기
with open("mystery_books_10000.json", "r", encoding="utf-8") as f:
    books_data = json.load(f)

# 📌 데이터프레임 변환
df = pd.DataFrame(books_data)

# 📌 1️⃣ ISBN 기준 중복 제거
df.drop_duplicates(subset=["isbn"], inplace=True)

# 📌 2️⃣ 줄거리(description)이 없는 책 제거
df = df[df["description"].str.strip() != ""]

# 📌 3️⃣ 특수문자 제거 (HTML 태그, 공백 등)
df["description"] = df["description"].apply(lambda x: re.sub(r"<.*?>", "", x).strip())  # HTML 태그 제거
df["title"] = df["title"].apply(lambda x: re.sub(r"\[.*?\]|\(.*?\)", "", x).strip())  # 불필요한 괄호 제거

# 📌 4️⃣ 정제된 데이터 저장
cleaned_books = df.to_dict(orient="records")

with open("clean_mystery_books.json", "w", encoding="utf-8") as f:
    json.dump(cleaned_books, f, ensure_ascii=False, indent=4)

# 📌 정제된 데이터 확인

df.head()
print(f"✅ 정제 완료! 중복 제거 후 {len(df)}권 저장 완료!")


✅ 정제 완료! 중복 제거 후 5572권 저장 완료!


In [1]:
import requests
import json
import os
import time

# ✅ 네이버 API 환경변수 사용
CLIENT_ID = os.getenv('X_NAVER_CLIENT_ID')
CLIENT_SECRET = os.getenv('X_NAVER_CLIENT_SECRET')

# ✅ 정제된 책 데이터 로드
with open("clean_mystery_books.json", "r", encoding="utf-8") as f:
    books_data = json.load(f)

# ✅ 네이버 블로그 검색 API 요청 함수 (429 오류 방지)
def search_blog_reviews(query, display=5, retries=3):
    url = "https://openapi.naver.com/v1/search/blog.json"
    headers = {
        "X-Naver-Client-Id": CLIENT_ID,
        "X-Naver-Client-Secret": CLIENT_SECRET
    }
    params = {
        "query": query,
        "display": display,
        "sort": "sim"
    }

    for attempt in range(retries):  # ✅ 최대 3번 재시도
        response = requests.get(url, headers=headers, params=params)

        if response.status_code == 200:
            return response.json()["items"]
        
        elif response.status_code == 429:
            wait_time = (attempt + 1) * 5  # ✅ 5초, 10초, 15초 대기
            print(f"🚨 429 오류: 요청 속도 제한 초과! ({query}) - {attempt + 1}번째 재시도 후 {wait_time}초 대기...")
            time.sleep(wait_time)

        else:
            print(f"❌ API 요청 실패: {response.status_code}, 검색어: {query}")
            return []

    return []  # 3번 재시도 후에도 실패하면 빈 리스트 반환

# ✅ 순차적 크롤링 실행
book_reviews = {}

for index, book in enumerate(books_data, 1):
    title = book["title"]
    reviews = search_blog_reviews(title)
    
    if reviews:
        book_reviews[title] = [review["description"] for review in reviews]

    print(f"📖 {index}/{len(books_data)} - '{title}' 리뷰 수집 완료! ({len(reviews)}개)")
    
    time.sleep(1.5)  # ✅ API 요청 간격 조정 (429 방지)

# ✅ 크롤링된 블로그 리뷰를 JSON 파일로 저장
with open("blog_reviews_sequential.json", "w", encoding="utf-8") as f:
    json.dump(book_reviews, f, ensure_ascii=False, indent=4)

print(f"✅ 총 {len(book_reviews)}권의 블로그 리뷰 저장 완료!")


📖 1/5572 - '당신이 누군가를 죽였다' 리뷰 수집 완료! (5개)
📖 2/5572 - '용의자 X의 헌신' 리뷰 수집 완료! (5개)
📖 3/5572 - '악의' 리뷰 수집 완료! (5개)
📖 4/5572 - '비정근' 리뷰 수집 완료! (5개)
📖 5/5572 - '셜록 홈스의 모험' 리뷰 수집 완료! (5개)
📖 6/5572 - '리버 2' 리뷰 수집 완료! (5개)
📖 7/5572 - '리버 1' 리뷰 수집 완료! (5개)
📖 8/5572 - '미로장의 참극' 리뷰 수집 완료! (5개)
📖 9/5572 - '얼음 속의 여인' 리뷰 수집 완료! (5개)
📖 10/5572 - '성소의 참새' 리뷰 수집 완료! (5개)
📖 11/5572 - '귀신 들린 아이' 리뷰 수집 완료! (5개)
📖 12/5572 - '죽은 자의 몸값' 리뷰 수집 완료! (5개)
📖 13/5572 - '아이 윌 파인드 유' 리뷰 수집 완료! (5개)
📖 14/5572 - '고행의 순례자' 리뷰 수집 완료! (5개)
📖 15/5572 - '캐드펠 수사 시리즈' 리뷰 수집 완료! (5개)
📖 16/5572 - '청과 부동명왕' 리뷰 수집 완료! (5개)
📖 17/5572 - '아서 코난 도일, 선상 미스터리 단편 컬렉션' 리뷰 수집 완료! (5개)
📖 18/5572 - '아름답고 위험한 이름, 비너스' 리뷰 수집 완료! (5개)
📖 19/5572 - '시체 한 구가 더 있다' 리뷰 수집 완료! (5개)
📖 20/5572 - '수도사의 두건' 리뷰 수집 완료! (5개)
📖 21/5572 - '세인트자일스의 나환자' 리뷰 수집 완료! (5개)
📖 22/5572 - '성 베드로 축일' 리뷰 수집 완료! (5개)
📖 23/5572 - '에도가와 란포 기담집' 리뷰 수집 완료! (5개)
📖 24/5572 - '유골에 대한 기이한 취향' 리뷰 수집 완료! (5개)
📖 25/5572 - '마녀와의 7일' 리뷰 수집 완료! (5개)
📖 26/5572 - '추리소설가의 살인사건' 리뷰 수집 완료! 

KeyError: 'items'

In [ ]:
import requests
import json
import os
import time
from urllib3.exceptions import ProtocolError
from requests.exceptions import ConnectionError

# ✅ 네이버 API 환경변수 사용
CLIENT_ID = os.getenv('X_NAVER_CLIENT_ID')
CLIENT_SECRET = os.getenv('X_NAVER_CLIENT_SECRET')

# ✅ 정제된 책 데이터 로드
with open("clean_mystery_books.json", "r", encoding="utf-8") as f:
    books_data = json.load(f)

# ✅ 네이버 블로그 검색 API 요청 함수 (429 오류 & RemoteDisconnected 예외 처리 추가)
def search_blog_reviews(query, display=5, retries=5):
    url = "https://openapi.naver.com/v1/search/blog.json"
    headers = {
        "X-Naver-Client-Id": CLIENT_ID,
        "X-Naver-Client-Secret": CLIENT_SECRET
    }
    params = {
        "query": query,
        "display": display,
        "sort": "sim"
    }



    for attempt in range(retries):  # ✅ 최대 5번 재시도
        try:
            response = requests.get(url, headers=headers, params=params, timeout=10)

            if response.status_code == 200:
                data = response.json()
                if "items" in data:
                    return data["items"]
                else:
                    print(f"⚠️ '{query}' 검색 결과 없음 (응답에 'items' 없음)")
                    return []

            elif response.status_code == 429:
                wait_time = (attempt + 1) * 5  # ✅ 5초, 10초, 15초, 20초 대기
                print(f"🚨 429 오류: 요청 속도 제한 초과! ({query}) - {attempt + 1}번째 재시도 후 {wait_time}초 대기...")
                time.sleep(wait_time)

            else:
                print(f"❌ API 요청 실패: {response.status_code}, 검색어: {query}")
                return []

        except (requests.exceptions.ConnectionError, requests.exceptions.Timeout, ProtocolError) as e:
            wait_time = (attempt + 1) * 5
            print(f"⚠️ 연결 오류: {query} ({str(e)}) - {attempt + 1}번째 재시도 후 {wait_time}초 대기...")
            time.sleep(wait_time)

    return []  # 5번 재시도 후에도 실패하면 빈 리스트 반환

# ✅ 순차적 크롤링 실행 (예외 처리 추가)
book_reviews = {}

for index, book in enumerate(books_data, 1):
    title = book.get("title", "제목 없음")  # ✅ title 키가 없을 경우 대비

    reviews = search_blog_reviews(title)
    
    if reviews:
        book_reviews[title] = [review["description"] for review in reviews]

    print(f"📖 {index}/{len(books_data)} - '{title}' 리뷰 수집 완료! ({len(reviews)}개)")
    
    time.sleep(3)  # ✅ API 요청 간격 증가 (429 & RemoteDisconnected 방지)

# ✅ 크롤링된 블로그 리뷰를 JSON 파일로 저장
with open("blog_reviews_sequential.json", "w", encoding="utf-8") as f:
    json.dump(book_reviews, f, ensure_ascii=False, indent=4)

print(f"✅ 총 {len(book_reviews)}권의 블로그 리뷰 저장 완료!")


📖 1/5572 - '당신이 누군가를 죽였다' 리뷰 수집 완료! (5개)
📖 2/5572 - '용의자 X의 헌신' 리뷰 수집 완료! (5개)
📖 3/5572 - '악의' 리뷰 수집 완료! (5개)
📖 4/5572 - '비정근' 리뷰 수집 완료! (5개)
📖 5/5572 - '셜록 홈스의 모험' 리뷰 수집 완료! (5개)
📖 6/5572 - '리버 2' 리뷰 수집 완료! (5개)
📖 7/5572 - '리버 1' 리뷰 수집 완료! (5개)
📖 8/5572 - '미로장의 참극' 리뷰 수집 완료! (5개)
📖 9/5572 - '얼음 속의 여인' 리뷰 수집 완료! (5개)
📖 10/5572 - '성소의 참새' 리뷰 수집 완료! (5개)
📖 11/5572 - '귀신 들린 아이' 리뷰 수집 완료! (5개)
📖 12/5572 - '죽은 자의 몸값' 리뷰 수집 완료! (5개)
📖 13/5572 - '아이 윌 파인드 유' 리뷰 수집 완료! (5개)
📖 14/5572 - '고행의 순례자' 리뷰 수집 완료! (5개)
📖 15/5572 - '캐드펠 수사 시리즈' 리뷰 수집 완료! (5개)
📖 16/5572 - '청과 부동명왕' 리뷰 수집 완료! (5개)
📖 17/5572 - '아서 코난 도일, 선상 미스터리 단편 컬렉션' 리뷰 수집 완료! (5개)
📖 18/5572 - '아름답고 위험한 이름, 비너스' 리뷰 수집 완료! (5개)
📖 19/5572 - '시체 한 구가 더 있다' 리뷰 수집 완료! (5개)
📖 20/5572 - '수도사의 두건' 리뷰 수집 완료! (5개)
📖 21/5572 - '세인트자일스의 나환자' 리뷰 수집 완료! (5개)
📖 22/5572 - '성 베드로 축일' 리뷰 수집 완료! (5개)
📖 23/5572 - '에도가와 란포 기담집' 리뷰 수집 완료! (5개)
📖 24/5572 - '유골에 대한 기이한 취향' 리뷰 수집 완료! (5개)
📖 25/5572 - '마녀와의 7일' 리뷰 수집 완료! (5개)
📖 26/5572 - '추리소설가의 살인사건' 리뷰 수집 완료! 

In [ ]:
import requests
import json
import os
import time
import re

# ✅ 네이버 API 환경변수 사용
CLIENT_ID = os.getenv('X_NAVER_CLIENT_ID')
CLIENT_SECRET = os.getenv('X_NAVER_CLIENT_SECRET')

# ✅ 정제된 책 데이터 로드
with open("clean_mystery_books.json", "r", encoding="utf-8") as f:
    books_data = json.load(f)

# ✅ 블로그 리뷰 필터링 함수 (소설 관련 단어 포함된 경우만 저장)
def filter_valid_reviews(reviews):
    valid_reviews = []
    allowed_keywords = ["책", "소설", "리뷰", "작품", "추천", "줄거리", "미스터리", "추리", "범죄", "작가", "스토리"]
    
    for review in reviews:
        title = review["title"]
        summary = review["summary"]

        if any(keyword in title for keyword in allowed_keywords) or any(keyword in summary for keyword in allowed_keywords):
            valid_reviews.append(review)

    return valid_reviews

# ✅ 불필요한 출판 정보 제거 함수
def clean_description(description):
    description = re.sub(r"(출판[:\s]?\S+|저자[:\s]?\S+|발매[:\s]?\d{4}.\d{2}.\d{2})", "", description)
    description = re.sub(r"<.*?>", "", description)  # HTML 태그 제거
    return description.strip()

# ✅ 소설 관련 키워드 등장 횟수 체크
def is_relevant_review(summary):
    keywords = ["책", "소설", "추리", "미스터리", "스토리", "범죄", "작가", "줄거리"]
    score = sum(summary.count(keyword) for keyword in keywords)
    return score >= 2

# ✅ 블로그 크롤링 실행
book_reviews = {}

for index, book in enumerate(books_data, 1):
    title = book.get("title", "제목 없음")

    # ✅ 네이버 블로그 검색 API 호출
    reviews = search_blog_reviews(title)

    # ✅ 리뷰 필터링 적용
    cleaned_reviews = []
    for review in reviews:
        clean_summary = clean_description(review["summary"])
        if is_relevant_review(clean_summary):
            cleaned_reviews.append({"title": review["title"], "summary": clean_summary, "link": review["link"]})

    # ✅ 필터링된 데이터 저장
    if cleaned_reviews:
        book_reviews[title] = cleaned_reviews

    print(f"📖 {index}/{len(books_data)} - '{title}' 리뷰 수집 완료! ({len(cleaned_reviews)}개)")
    
    time.sleep(1.5)  # ✅ API 요청 간격 조정

# ✅ 크롤링된 블로그 리뷰를 JSON 파일로 저장
with open("filtered_blog_reviews.json", "w", encoding="utf-8") as f:
    json.dump(book_reviews, f, ensure_ascii=False, indent=4)

print(f"✅ 총 {len(book_reviews)}권의 블로그 리뷰 저장 완료!")


In [7]:
import requests
import json
import os
import time
import re

# ✅ 네이버 API 환경변수 사용
CLIENT_ID = os.getenv('X_NAVER_CLIENT_ID')
CLIENT_SECRET = os.getenv('X_NAVER_CLIENT_SECRET')

# ✅ 정제된 책 데이터 로드
with open("clean_mystery_books.json", "r", encoding="utf-8") as f:
    books_data = json.load(f)

# ✅ 네이버 블로그 검색 API 요청 함수 (429 오류 방지 + 응답이 없을 경우 빈 리스트 반환)
def search_blog_reviews(query, display=5, retries=3):
    url = "https://openapi.naver.com/v1/search/blog.json"
    headers = {
        "X-Naver-Client-Id": CLIENT_ID,
        "X-Naver-Client-Secret": CLIENT_SECRET
    }
    params = {
        "query": query,
        "display": display,
        "sort": "sim"
    }

    for attempt in range(retries):  # ✅ 최대 3번 재시도
        response = requests.get(url, headers=headers, params=params)

        if response.status_code == 200:
            data = response.json()
            if "items" in data:
                return data["items"]  # ✅ API 응답이 있을 경우 리스트 반환
            else:
                print(f"⚠️ '{query}' 검색 결과 없음")
                return []  # ✅ 응답에 'items'가 없을 경우 빈 리스트 반환

        elif response.status_code == 429:
            wait_time = (attempt + 1) * 5  # ✅ 5초, 10초, 15초 대기
            print(f"🚨 429 오류: 요청 속도 제한 초과! ({query}) - {attempt + 1}번째 재시도 후 {wait_time}초 대기...")
            time.sleep(wait_time)

        else:
            print(f"❌ API 요청 실패: {response.status_code}, 검색어: {query}")
            return []  # ✅ API 요청 실패 시 빈 리스트 반환

    return []  # ✅ 3번 재시도 후에도 실패하면 빈 리스트 반환

# ✅ 블로그 리뷰 필터링 함수 (소설 관련 단어 포함된 경우만 저장)
def filter_valid_reviews(reviews):
    valid_reviews = []
    allowed_keywords = ["책", "소설", "리뷰", "작품", "추천", "줄거리", "미스터리", "추리", "범죄", "작가", "스토리"]
    
    for review in reviews:
        title = review["title"]
        summary = review["summary"]

        if any(keyword in title for keyword in allowed_keywords) or any(keyword in summary for keyword in allowed_keywords):
            valid_reviews.append(review)

    return valid_reviews

# ✅ 불필요한 출판 정보 제거 함수
def clean_description(description):
    description = re.sub(r"(출판[:\s]?\S+|저자[:\s]?\S+|발매[:\s]?\d{4}.\d{2}.\d{2})", "", description)
    description = re.sub(r"<.*?>", "", description)  # HTML 태그 제거
    return description.strip()

# ✅ 소설 관련 키워드 등장 횟수 체크
def is_relevant_review(summary):
    keywords = ["책", "소설", "추리", "미스터리", "스토리", "범죄", "작가", "줄거리"]
    score = sum(summary.count(keyword) for keyword in keywords)
    return score >= 2

# ✅ 블로그 크롤링 실행 (처음 20개 책만 크롤링)
book_reviews = {}

for index, book in enumerate(books_data[:20], 1):  # 🔥 20개만 크롤링
    title = book.get("title", "제목 없음")

    # ✅ 네이버 블로그 검색 API 호출
    reviews = search_blog_reviews(title)  # ✅ 이제 빈 리스트([])라도 항상 반환됨

    # ✅ 리뷰 필터링 적용
    cleaned_reviews = []
    for review in reviews:
        clean_summary = clean_description(review["description"])  # ✅ "summary" → "description" 수정
        if is_relevant_review(clean_summary):
            cleaned_reviews.append({"title": review["title"], "summary": clean_summary, "link": review["link"]})

    # ✅ 필터링된 데이터 저장
    if cleaned_reviews:
        book_reviews[title] = cleaned_reviews

    print(f"📖 {index}/20 - '{title}' 리뷰 수집 완료! ({len(cleaned_reviews)}개)")
    
    time.sleep(1.5)  # ✅ API 요청 간격 조정

# ✅ 크롤링된 블로그 리뷰를 JSON 파일로 저장
with open("filtered_blog_reviews_20.json", "w", encoding="utf-8") as f:
    json.dump(book_reviews, f, ensure_ascii=False, indent=4)

print(f"✅ 총 {len(book_reviews)}권의 블로그 리뷰 저장 완료! (20개 중)")


📖 1/20 - '당신이 누군가를 죽였다' 리뷰 수집 완료! (1개)
📖 2/20 - '용의자 X의 헌신' 리뷰 수집 완료! (3개)
📖 3/20 - '악의' 리뷰 수집 완료! (0개)
📖 4/20 - '비정근' 리뷰 수집 완료! (2개)
📖 5/20 - '셜록 홈스의 모험' 리뷰 수집 완료! (1개)
📖 6/20 - '리버 2' 리뷰 수집 완료! (0개)
📖 7/20 - '리버 1' 리뷰 수집 완료! (1개)
📖 8/20 - '미로장의 참극' 리뷰 수집 완료! (2개)
📖 9/20 - '얼음 속의 여인' 리뷰 수집 완료! (2개)
📖 10/20 - '성소의 참새' 리뷰 수집 완료! (1개)
📖 11/20 - '귀신 들린 아이' 리뷰 수집 완료! (1개)
📖 12/20 - '죽은 자의 몸값' 리뷰 수집 완료! (0개)
📖 13/20 - '아이 윌 파인드 유' 리뷰 수집 완료! (0개)
📖 14/20 - '고행의 순례자' 리뷰 수집 완료! (2개)
📖 15/20 - '캐드펠 수사 시리즈' 리뷰 수집 완료! (3개)
📖 16/20 - '청과 부동명왕' 리뷰 수집 완료! (0개)
📖 17/20 - '아서 코난 도일, 선상 미스터리 단편 컬렉션' 리뷰 수집 완료! (5개)
📖 18/20 - '아름답고 위험한 이름, 비너스' 리뷰 수집 완료! (0개)
📖 19/20 - '시체 한 구가 더 있다' 리뷰 수집 완료! (1개)
📖 20/20 - '수도사의 두건' 리뷰 수집 완료! (1개)
✅ 총 14권의 블로그 리뷰 저장 완료! (20개 중)


In [ ]:
import requests
import os
import json

# ✅ 알라딘 API 키 (환경변수에서 가져오기)
TTBKey = os.getenv('TTBKey')
# ✅ ISBN을 통한 도서 정보 조회 함수
def get_book_info_by_isbn(isbn):
    url = "http://www.aladin.co.kr/ttb/api/ItemLookUp.aspx"
    params = {
        "TTBKey": TTBKey,  # ✅ 알라딘 API 키
        "ItemId": isbn,  # ✅ ISBN을 ItemId로 전달 (isbn13 → isbn 수정)
        "ItemIdType": "ISBN13",  # ✅ ISBN13 사용
        "Cover": "Big",  # ✅ 커버 이미지 크기
        "Output": "js",  # ✅ JSON 형식 응답
        "Version": "20131101",  # ✅ API 버전
        "OptResult":["Story","fulldescription2"]
    }

    response = requests.get(url, params=params)
    if response.status_code == 200:
        try:
            data = response.json()
            if "item" in data:
                return data["item"]
            else:
                print(f"⚠️ '{isbn}'에 대한 검색 결과가 없습니다.")
                return None
        except json.JSONDecodeError:
            print(f"❌ JSON 디코딩 오류: {response.text}")
            return None
    else:
        print(f"❌ API 요청 실패: {response.status_code}, ISBN: {isbn}")
        return None

# ✅ 예제 실행 (올바른 변수 사용)
isbn = "9791170611561"  # ✅ 검색할 도서 ISBN13
book_info = get_book_info_by_isbn(isbn)

if book_info:
    for book in book_info:
        print(f"📖 제목: {book.get('title', '제목 없음')}")
        print(f"📜 줄거리: {book.get('description', '줄거리 없음')}")
        print(f"✍️ 저자: {book.get('author', '저자 정보 없음')}")
        print(f"🏢 출판사: {book.get('publisher', '출판사 정보 없음')}")
        print(f"📅 출간일: {book.get('pubDate', '출간일 없음')}")
        print(f"📘 표지 이미지: {book.get('cover', '이미지 없음')}")
        print(f"🔗 링크: {book.get('link', '링크 없음')}\n")
else:
    print("⚠️ 도서 정보를 가져올 수 없습니다.")
    




📖 제목: 당신이 누군가를 죽였다
📜 줄거리: 일본 최고의 베스트셀러 작가 히가시노 게이고의 장편소설『당신이 누군가를 죽였다』가 출간되었다. 작품은 장르문학계의 거장인 작가가 101번째 작품을 맞아 추리소설의 원점으로 돌아가 ‘황금시대 미스터리’의 매력을 유감없이 발휘한 걸작으로 평단과 독자의 호평을 받고 있다.
✍️ 저자: 히가시노 게이고 (지은이), 최고은 (옮긴이)
🏢 출판사: 북다
📅 출간일: 2024-07-23
📘 표지 이미지: https://image.aladin.co.kr/product/34292/25/cover200/k692932177_1.jpg
🔗 링크: http://www.aladin.co.kr/shop/wproduct.aspx?ItemId=342922559&amp;partner=openAPI&amp;start=api



In [38]:
import json
import re

# ✅ 중국어 & 일본어 감지하는 정규표현식 (Unicode 범위)
chinese_japanese_pattern = re.compile(r"[\u3040-\u30FF\u4E00-\u9FFF]")  # 히라가나, 가타카나, 한자

# ✅ JSON 파일 로드
with open("clean_mystery_books.json", "r", encoding="utf-8") as f:
    books_data = json.load(f)

# ✅ 필터링 함수 (제목과 설명에서 중국어나 일본어 포함 시 제거)
def contains_chinese_or_japanese(text):
    return bool(chinese_japanese_pattern.search(text))

# ✅ 필터링된 도서 리스트 생성
filtered_books = [
    book for book in books_data
    if not contains_chinese_or_japanese(book.get("title", "")) and not contains_chinese_or_japanese(book.get("description", ""))
]

# ✅ 필터링 결과 저장
with open("filtered_mystery_books.json", "w", encoding="utf-8") as f:
    json.dump(filtered_books, f, ensure_ascii=False, indent=4)

print(f"✅ 필터링 완료! 총 {len(filtered_books)}권의 도서가 저장되었습니다.")


✅ 필터링 완료! 총 5328권의 도서가 저장되었습니다.


In [17]:
import json
import re

# ✅ 필터링할 키워드 리스트
exclude_keywords = ["실무론","학술정보","범죄론",
    "직무적성검사", "하브루타", "추리퀴즈", "약사의 혼잣말", "학습", "밥스 패밀리", "신비아파트", "수학 탐정스", "어린이", 
    "미스터리 수학", "범죄심리학", "이론 총서", "인적성검사", "추리논증", "LEET", "법학적성시험", "범죄학", "형사법", "형법", 
    "수사학", "법학", "법조", "검찰", "경찰", "귀신 보는 추리 탐정", "고양이 탐정 윈스턴", "전공", "수험서", "시험", "문제집", 
    "논문", "형사정책", "카카오", "직소퍼즐", "엉덩이탐정", "엉덩이 탐정", "스티커 탐정", "초등탐정", "수상한 전학생", "맥거크 탐정단"
]

# ✅ JSON 파일 로드
with open("clean_mystery_books.json", "r", encoding="utf-8") as f:
    books_data = json.load(f)

# ✅ 필터링 함수 (제목, 설명, 저자, 출판사에서 특정 키워드 포함 시 제거)
def is_irrelevant_book(book):
    title = book.get("title", "").lower()
    description = book.get("description", "").lower()
    author = book.get("author", "").lower()  # 🔹 저자 추가
    publisher = book.get("publisher", "").lower()  # 🔹 출판사 추가

    # ✅ 키워드 포함 여부 확인 (정규표현식 제거, 부분 포함 검사로 변경)
    for keyword in exclude_keywords:
        if keyword in title or keyword in description or keyword in author or keyword in publisher:
            return True  # 🚨 해당 키워드가 포함된 책은 제거
    return False

# ✅ 필터링된 도서 리스트 생성
filtered_books = [book for book in books_data if not is_irrelevant_book(book)]

# ✅ 필터링 결과 저장
with open("filtered_mystery_books.json", "w", encoding="utf-8") as f:
    json.dump(filtered_books, f, ensure_ascii=False, indent=4)

print(f"✅ 필터링 완료! 총 {len(filtered_books)}권의 도서가 저장되었습니다.")


✅ 필터링 완료! 총 3980권의 도서가 저장되었습니다.
